In [45]:
import torch
import glob
from __future__ import print_function #print from python3
from IPython.display import display


In [46]:
def readImages():
    #dossier ou se trouvent les images :
    folder = "/home/data/collection/GUIMUTEIC/CLICIDE/CLICIDEMAX/"
    
    #on parcourt le dossier :
    listImages = glob.glob(folder+'*.JPG')
    listImagesTest = glob.glob(folder+'test/*.JPG')
    
    #extraction de la classe des images
    listImagesClass = []
    for im in listImages:
        c = im.split("/")[-1]
        c = c.split("-")[0]
        listImagesClass.append(c)
    #print(len(listImages) == len(listImagesClass))
    listImagesTestClass = []
    for im in listImagesTest:
        c = im.split("/")[-1]
        c = c.split("-")[0]
        listImagesTestClass.append(c)
    
    return (listImages, listImagesClass), (listImagesTest, listImagesTestClass)

In [47]:
train, test = readImages()

In [48]:
mean = (0.426, 0.426, 0.418)
std_dev = (0.200, 0.176, 0.170)
print(mean, std_dev)

(0.426, 0.426, 0.418) (0.2, 0.176, 0.17)


In [49]:
import torch.nn as nn
import torchvision.models as models
from PIL import Image

In [50]:
model1 = models.alexnet(pretrained=True).eval()
print(model1)

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)


In [51]:
import torchvision.transforms as transforms
inputs = torch.Tensor(2,3,224,224)
imageOpen = Image.open(train[0][0])
imageOpen = imageOpen.resize((224,224), Image.BILINEAR)

imageTransformation = transforms.Compose( 
                        (transforms.ToTensor(),
                        transforms.Normalize(mean, std_dev) )
                        )

inputs[0] = imageTransformation(imageOpen)
inputs[1] = imageTransformation(Image.open(train[0][10]).resize((224,224), Image.BILINEAR))

In [52]:
from torch.autograd import Variable
outputs = model1(Variable(inputs))

In [53]:
print(outputs)
print(len(outputs[0]))

Variable containing:
  2.9713  -1.2717   0.9710  ...   -0.9601   0.0638   1.4522
  6.7962   0.5667   0.5651  ...    2.1066   0.7941  -3.1658
[torch.FloatTensor of size 2x1000]

1000


In [54]:
model1.train()

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)

In [55]:
class maxnet(nn.Module):
    def __init__(self, nbClass=464):
        super(maxnet, self).__init__()
        self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1)),
                nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                nn.ReLU(True),
                nn.MaxPool2d((3, 3), stride=(2, 2), dilation=(1, 1))
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, nbClass),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [67]:
import torch.optim as optim
mod = maxnet().train()

criterion = nn.loss.CrossEntropyLoss()
optimizer = optim.SGD(mod.parameters(), lr=0.01, momentum=0.9)

imageTransformation = transforms.Compose( 
                                (transforms.ToTensor(),
                                transforms.Normalize(mean, std_dev) )
                                )

listLabel = set()
for el in train[1]:
    if not 'wall' in el:
        listLabel.add(el)

In [71]:
print(len(listLabel))
listLabel = sorted(list(listLabel))
print(listLabel)

464
['10A', '10B', '10C', '10D', '10E', '10F', '10G', '10H', '10I', '10J', '11A', '11B', '11C', '11D', '11E', '11F', '11G', '11H', '11I', '11J', '11K', '11L', '11M', '11N', '11O', '12A', '12B', '12C', '12D', '12E', '12F', '12G', '12H', '12I', '12J', '12K', '12O', '12P', '13A', '13C', '13D', '13E', '13G', '13H', '14B', '14C', '14D', '14E', '14F', '14G', '14H', '14I', '14J', '14K', '14L', '14M', '15A', '15B', '15C', '15D', '15E', '15F', '15G', '15J', '15K', '15L', '15N', '15O', '15P', '15Q', '15R', '15S', '15T', '16A', '16B', '16C', '16D', '16E', '16F', '16G', '16H', '16I', '16J', '16K', '16L', '16M', '16N', '16O', '16P', '16Q', '17A', '17B', '17C', '17D', '17E', '17F', '17G', '17H', '17I', '17J', '17K', '17L', '17M', '17N', '1A', '1B', '1C', '1D', '1E', '1F', '1G', '1H', '1I', '1J', '1K', '1L', '1M', '1N', '1O', '1P', '1Q', '1R', '20A', '20B', '20C', '20D', '20E', '20F', '20G', '20H', '20I', '20J', '20K', '20L', '20M', '20N', '21A', '21B', '21C', '21D', '21E', '21F', '21G', '21H', '21J'

In [58]:
from torch.autograd import Variable

In [63]:
optimizer = optim.SGD(mod.parameters(), lr=0.1, momentum=0.9)
for epoch in range(10):
    running_loss = 0.0
    #chaque epoch parcourt la collection
    for i, im in enumerate(train[0]):
        inputs = torch.Tensor(1,3,224,224)
        imageOpen = Image.open(im).resize((224,224), Image.BILINEAR)
        inputs[0] = imageTransformation(imageOpen)
        inputs = Variable(inputs)
        
        outputs = mod(inputs)
        
        lab = Variable(torch.LongTensor([listLabel.index(train[1][0])]))
        optimizer.zero_grad()
        loss = criterion(outputs, lab)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i % 10 == 9: # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
        
        

AttributeError: 'set' object has no attribute 'index'